# CNN and LRP on Social Physical brain

This notebook shows how saliency maps are computed for social and physical pain brain dataset. We will first train a model and then apply LRP methods. 

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.simplefilter('ignore')

In [13]:
%matplotlib inline  

import h5py
import imp
import numpy as np
import os


import keras
import keras.backend
import keras.models

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)


from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras import regularizers
from keras import backend as K

import sys
sys.path.insert(0, '../../')

import innvestigate
import innvestigate.utils as iutils
import innvestigate.utils.visualizations as ivis

# Use utility libraries to focus on relevant iNNvestigate routines.
eutils = imp.load_source("utils", "../utils.py")






## Variable
Set the variable for making model and the analysis.

selection_index_LRP : if you want to see the LRP results with the 1 for Social score and 0 for Physical score and None for the most model prediction score.

In [ ]:
mini_batch_size = 128
data_file_name = '../data/lrp_data/mini_example_social_physical_masked_cross.hdf5'
label_to_class_name = [str(i) for i in range(2)]
selection_index_LRP = 0 # 0, 1, None

## Model
The next part trains and evaluates a CNN.

In [5]:
# make CNN model with 6 layers, Flatten() is inserted between CNN and fully connected layers.
def make_custom_model_cnn_2D():
       

    model = Sequential() 
    model.add(Conv2D(8, (3,3), kernel_initializer='he_normal', padding='same', input_shape=(68,95,79)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))


    model.add(Conv2D(16, (3,3), kernel_initializer='he_normal', padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(32, (3,3), kernel_initializer='he_normal', padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))

    model.add(Conv2D(64, (3,3), kernel_initializer='he_normal', padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))


    model.add(Flatten()) 
    model.add(Dense(128, kernel_initializer='he_normal'))
    model.add(Activation('relu'))
    
    model.add(Dense(2, kernel_initializer='he_normal'))
    model.add(Activation('linear'))
    
    model.add(Activation('softmax'))
    
    adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

    return model

In [14]:
# Functions for creating & training model
def train_model():

    """
    train model with data in every cross_i. Everytime model trained, get_result and generate_sequences are called
    """
    # Option : when you don't have enough GPUs
#     with tf.device('/cpu:0'):

    with h5py.File(data_file_name, "r") as data:

        for i in range(0,3):  # i is the cross number 59

            print('this is ith iter : ' , i)

            tr_data_X_name = 'cross_'+str(i+1)+ '_X'+'_train'
            tr_data_y_name = 'cross_'+str(i+1)+ '_y'+'_train'
            te_data_X_name = 'cross_'+str(i+1)+ '_X'+'_test'
            te_data_y_name = 'cross_'+str(i+1)+ '_y'+'_test'

            tr_data = {}
            tr_data['X_data'] = np.array(data[tr_data_X_name])
            tr_data['y_data'] = np.array(data[tr_data_y_name])
            
            te_data = {}
            te_data['X_data'] = np.array(data[te_data_X_name])
            te_data['y_data'] = np.array(data[te_data_y_name])
            
            tr_data['y_data'] = keras.utils.to_categorical(tr_data['y_data'], 2)
            te_data['y_data'] = keras.utils.to_categorical(te_data['y_data'], 2)
   
            training_sample_count = tr_data['X_data'].shape[0]


            # Initialize and compile the model
            model = make_custom_model_cnn_2D()
            model.compile(loss="categorical_crossentropy",
                          optimizer=Adam(),
                          metrics=["accuracy"])
            history = model.fit(tr_data['X_data'],tr_data['y_data'],
                                batch_size=mini_batch_size,
                                epochs=20,
                                verbose=1)
            score = model.evaluate(te_data['X_data'], te_data['y_data'], verbose=0)
            return model,score


In [15]:
# Create & train model
model,score = train_model()

this is ith iter :  0
Epoch 1/20
64/64 [==============================] - 1s 18ms/step - loss: 1.4811 - acc: 0.4688
Epoch 2/20
64/64 [==============================] - 0s 4ms/step - loss: 0.7014 - acc: 0.6406
Epoch 3/20
64/64 [==============================] - 0s 4ms/step - loss: 0.7998 - acc: 0.5312
Epoch 4/20
64/64 [==============================] - 0s 4ms/step - loss: 0.5704 - acc: 0.7188
Epoch 5/20
64/64 [==============================] - 0s 4ms/step - loss: 0.4441 - acc: 0.7344
Epoch 6/20
64/64 [==============================] - 0s 4ms/step - loss: 0.4814 - acc: 0.6250
Epoch 7/20
64/64 [==============================] - 0s 4ms/step - loss: 0.3890 - acc: 0.8438
Epoch 8/20
64/64 [==============================] - 0s 4ms/step - loss: 0.3105 - acc: 0.9531
Epoch 9/20
64/64 [==============================] - 0s 4ms/step - loss: 0.2911 - acc: 0.9375
Epoch 10/20
64/64 [==============================] - 0s 4ms/step - loss: 0.2488 - acc: 0.9688
Epoch 11/20
64/64 [===========================

## Accuracy

In [8]:

val_acc = [] #List for save accuracy
with h5py.File(data_file_name, "r") as data:

    for i in range(0,3):  # i is the cross number 59

        print('this is ith iter : ' , i)

        te_data_X_name = 'cross_'+str(i+1)+ '_X'+'_test'
        te_data_y_name = 'cross_'+str(i+1)+ '_y'+'_test'

        te_data = {}
        te_data['X_data'] = np.array(data[te_data_X_name])
        te_data['y_data'] = np.array(data[te_data_y_name])
        
        te_data['y_data'] = keras.utils.to_categorical(te_data['y_data'], 2)
        

        score = model.evaluate(te_data['X_data'], te_data['y_data'], verbose=0)
        print(score[1])
        val_acc.append(score[1])

print("mean of validation accuracy",np.mean(val_acc))


this is ith iter :  0
0.8125
this is ith iter :  1
1.0
this is ith iter :  2
1.0
mean of validation accuracy 0.9375


## Analysis

set up a list of analysis methods used by `innvestigate.analyzer.create_analyzer(...)`, some optional parameters, a post processing choice for visualizing the computed analysis and a title for the figure to render. 
For a full list of methods refer to the dictionary `investigate.analyzer.analyzers`

In [9]:
# Configure analysis methods and properties
if selection_index_LRP == None:
    methods = [
    # NAME                    OPT.PARAMS                POSTPROC FXN               TITLE

    # Show input
    ("lrp.epsilon",           {"epsilon": 1,"neuron_selection_mode":"max_activation"},           ivis.heatmap,        "LRP-Epsilon"),
    ]
else:    
    methods = [
        # NAME                    OPT.PARAMS                POSTPROC FXN               TITLE

        # Show input
        ("lrp.epsilon",           {"epsilon": 1, "neuron_selection_mode" : "index"},           ivis.heatmap,        "LRP-Epsilon"),
    ]

The main loop below will now instantiate the analyzer objects based on the loaded/trained model and the analyzers' parameterizations above.

In [10]:
# Create model without trailing softmax
model_wo_softmax = iutils.keras.graph.model_wo_softmax(model)

# Create analyzers.
analyzers = []
for method in methods:
    
    analyzer = innvestigate.create_analyzer(method[0],        # analysis method identifier
                                            model_wo_softmax, # model without softmax output
                                            **method[1])      # optional analysis parameters

    # Some analyzers require training.

    analyzer.fit(data, batch_size=256, verbose=1)
    analyzers.append(analyzer)
    

Now we analyze each image with the different analyzers:

In [11]:
# %%capture

with h5py.File(data_file_name, "r") as data:

    for i in range(0,1):  # i is the cross number 59

        print('this is ith iter : ' , i)

        tr_data_X_name = 'cross_'+str(i+1)+ '_X'+'_test'
        tr_data_y_name = 'cross_'+str(i+1)+ '_y'+'_test'

        tr_data = {}
        tr_data['X_data'] = np.array(data[tr_data_X_name])
        tr_data['y_data'] = np.array(data[tr_data_y_name])
        
        test_data = tr_data['X_data']
        test_label = tr_data['y_data']

        n = 10
        test_images = list(zip(test_data[:n], test_label[:n]))

        analysis = np.zeros([len(test_images), len(analyzers), 68, 95, 3])
        text = []
        R=[]



        for p, (x, y) in enumerate(test_images):
            # Add batch axis.
            x = x[None, :, :, :]

            # Predict final activations, probabilites, and label.
            presm = model_wo_softmax.predict_on_batch(x)[0]
            prob = model.predict_on_batch(x)[0]
            y_hat = prob.argmax()


            # Save prediction info:
            text.append(("ground truth label : %s" % label_to_class_name[int(y)],    # ground truth label
                         "pre-softmax logits : %.2f" % presm.max(),             # pre-softmax logits
                         "probabilistic softmax output   : %.2f" % prob.max(),              # probabilistic softmax output  
                         "predicted label : %s" % label_to_class_name[y_hat] # predicted label
                        ))

            for aidx, analyzer in enumerate(analyzers):
                # Analyze.
                if selection_index_LRP == None:
                    a = analyzer.analyze(x)
                else:
                    a = analyzer.analyze(x, neuron_selection=selection_index_LRP)
                R.append(a) # Save natual R
                



        # Save Relevance as
        R_name = 'social_physical_brain_relevance_cross_'+str(i)+'.npy'
        np.save(R_name, R)
        print(text)

this is ith iter :  0
[('ground truth label : 0', 'pre-softmax logits : 3.68', 'probabilistic softmax output   : 1.00', 'predicted label : 0'), ('ground truth label : 0', 'pre-softmax logits : 6.79', 'probabilistic softmax output   : 1.00', 'predicted label : 0'), ('ground truth label : 0', 'pre-softmax logits : 3.61', 'probabilistic softmax output   : 1.00', 'predicted label : 0'), ('ground truth label : 0', 'pre-softmax logits : 3.40', 'probabilistic softmax output   : 1.00', 'predicted label : 0'), ('ground truth label : 0', 'pre-softmax logits : 2.13', 'probabilistic softmax output   : 0.75', 'predicted label : 1'), ('ground truth label : 0', 'pre-softmax logits : 3.48', 'probabilistic softmax output   : 0.99', 'predicted label : 0'), ('ground truth label : 0', 'pre-softmax logits : 2.33', 'probabilistic softmax output   : 0.96', 'predicted label : 0'), ('ground truth label : 0', 'pre-softmax logits : 2.51', 'probabilistic softmax output   : 0.97', 'predicted label : 0'), ('ground 